In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv('data/abrams.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6824 entries, 0 to 6823
Data columns (total 37 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            6824 non-null   int64  
 1   tweet_url                     6824 non-null   object 
 2   created_at                    6824 non-null   object 
 3   parsed_created_at             6824 non-null   object 
 4   user_screen_name              6824 non-null   object 
 5   text                          6824 non-null   object 
 6   tweet_type                    6824 non-null   object 
 7   coordinates                   1 non-null      object 
 8   hashtags                      261 non-null    object 
 9   media                         570 non-null    object 
 10  urls                          626 non-null    object 
 11  favorite_count                6824 non-null   int64  
 12  in_reply_to_screen_name       470 non-null    object 
 13  in_

In [4]:
# count number of different languages in data set
count_lang = df['lang'].unique()
print(len(count_lang), count_lang)

23 ['en' 'und' 'es' 'fr' 'pt' 'ca' 'sv' 'ro' 'cy' 'de' 'zh' 'no' 'in' 'tl'
 'iw' 'ht' 'da' 'cs' 'ar' 'fi' 'tr' 'pl' 'it']


In [5]:
# tweets are in 23 different languages. 
# I'll be working only with tweets in English
# so, I'll drop tweets in all other languages
df = df[df.lang == 'en']
df.shape

(6659, 37)

In [6]:
# drop unnecessary columns 
df = df.drop(['tweet_url', 'created_at', 'media', 'urls','in_reply_to_screen_name',
       'in_reply_to_status_id', 'in_reply_to_user_id', 'retweet_or_quote_id',
       'retweet_or_quote_screen_name', 'retweet_or_quote_user_id', 'source',
       'user_created_at', 'user_name', 'user_verified', 'user_friends_count', 'user_listed_count',
       'user_statuses_count', 'user_default_profile_image', 'user_description',
       'user_favourites_count', 'user_followers_count', 'coordinates', 'lang', 'user_location', 'user_time_zone', 'user_urls', 'place'], axis=1)

In [7]:
text = df['text'] 

In [8]:
# tokenize, remove stopwords, remove urls, lowercase, remove punctuation, remove numbers

import string
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk import pos_tag

stop = stopwords.words('english')

punc = list(set(string.punctuation))

def tokenizer(text):
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(text)
    return tokens

def remove_url(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r'', text)

def process_text(text):
    text = remove_url(text)
    text = tokenizer(text)
    text = [word.lower() for word in text]
    text = [re.sub('[0-9]+', '', word) for word in text]
    text = [word for word in text if word not in punc]
    text = [word for word in text if word not in stop]
    text = [each for each in text if len(each) > 1]
    text = [word for word in text if ' ' not in word]
     
    return text

In [9]:
df['processed_text'] = df['text'].apply(process_text)

In [10]:
pd.set_option('display.max_colwidth', -1)
df['processed_text'][:20]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


0     [stacey, abrams, brags, allowing, fake, voters, signature, verification]                                                                                                                                                   
1     [stacey, abrams, damn, thing]                                                                                                                                                                                              
2     [house, cards, shit, republicans, stole, election, abrams, abrams, plotted, took, washington]                                                                                                                              
3     [actually, happens, actually, win, discussion, stacey, abrams, saved, america]                                                                                                                                             
4     [chuck, schumer, stacey, abrams, years, team, many, women's, groups, black, women's, group

In [11]:
ready_for_pos = df['processed_text']

In [12]:
# part-of-speech tagging 
def pos_tagging(text):
    pos_tag = [pos_tag(word) for word in ready_for_pos]

df['pos_tagged'] = df.processed_text.apply(lambda x: pos_tag(x))

In [13]:
pos_tagged = df['pos_tagged']

In [14]:
# lemmatizing

from nltk.stem import WordNetLemmatizer 
wordnet = WordNetLemmatizer() 

lemmatized = [[wordnet.lemmatize(word[0]) for word in words] for words in pos_tagged]

In [15]:
df['lemmatized'] = lemmatized

In [16]:
df['final_docs'] = df['lemmatized'].apply(lambda x: " ".join(x))

In [17]:
df['final_docs'][3000:3020]

3083    good morning @staceyabrams stacey abrams                                                                                                                                                               
3084    guess good idea stacey abrams run senate instead helped biden two senator win revolutionized organizing put stake gop                                                                                  
3085    give stacey abrams fair fight new georgia project every single organizer made possible credit done georgia remarkable                                                                                  
3086    stacey abrams spent decade building democratic infrastructure georgia democrat move closer flipping georgia senate seat many see person responsible shifting political landscape                       
3087    stacey abrams destroyed election integrity georgia brian kemp sat watched happen georgia blue state stolen state                                                

In [18]:
final_docs = df['final_docs']

In [19]:
#create document term matrix with TFIDF

from sklearn.feature_extraction.text import TfidfVectorizer
# initial tuning of parameters
#set max_features to 2000 (specifies the number of most frequently occurring words for which we want to create feature vectors)
# set min_df to 5 (word must occur in at least 5 documents)
# set max_df to 0.85 (word must not occur in more than 85 percent of the documents) 

tfidfconverter = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.85, ngram_range=(1, 2), stop_words='english')  
doc_term_matrix_1 = tfidfconverter.fit_transform(df['final_docs'].values.astype('U'))

In [20]:
from sklearn.decomposition import NMF

nmf_model = NMF(n_components=10)
nmf_Z = nmf_model.fit_transform(doc_term_matrix_1)


In [21]:
from sklearn.decomposition import LatentDirichletAllocation
#  LDA model 
lda_model = LatentDirichletAllocation(n_components = 10, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(doc_term_matrix_1)

In [22]:
def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
 
print("LDA Model:")
print_topics(lda_model, tfidfconverter )
print("=" * 20)
 
print("NMF Model:")
print_topics(nmf_model, tfidfconverter )
print("=" * 20)
 


LDA Model:
Topic 0:
[('president', 31.99579936668867), ('mcconnell', 31.31803817465682), ('biden', 27.569885149012975), ('mitch mcconnell', 27.452868502004083), ('mitch', 26.77208849354789), ('american', 24.375810467677073), ('america', 22.14469200099454), ('power', 21.894561031410518), ('fact', 20.734354662031322), ('medal', 19.553887173266823)]
Topic 1:
[('stacey', 136.58723361525503), ('stacey abrams', 135.7136584841917), ('tweet', 125.5378676184316), ('carrying', 122.02203094684948), ('abrams tweet', 121.46089611237687), ('carrying democracy', 121.05831502880673), ('abrams carrying', 121.05831496535741), ('democracy', 104.42116132059934), ('absolutely', 36.64458886360368), ('day', 36.05101868999824)]
Topic 2:
[('state', 141.90970979301954), ('lead', 90.5573367919726), ('follow', 88.03895374755996), ('abrams state', 87.14168257983332), ('follow lead', 85.77980609457458), ('invest', 85.5181462702445), ('state stacey', 85.38181613985952), ('trust', 85.3647244065772), ('invest follow',

In [23]:
import pyLDAvis.sklearn

In [24]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, doc_term_matrix_1, tfidfconverter, mds='tsne')
panel

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
5      32.420952  8.115230   1       1        12.639200
7      0.648955   71.707100  2       1        11.914760
1      71.019684  9.971006   3       1        11.156322
3      3.785981   33.185135  4       1        11.073140
9     -42.666500 -9.222070   5       1        10.421421
6     -3.621689  -43.526516  6       1        10.046529
4     -3.632402  -4.171708   7       1        9.315782 
2     -36.584400  35.804413  8       1        8.830660 
0      44.391380  50.615383  9       1        8.080466 
8      40.235809 -31.591665  10      1        6.521721 , topic_info=                     Term        Freq       Total Category  logprob  loglift
1647  state                189.000000  189.000000  Default  30.0000  30.0000
306   carrying             102.000000  102.000000  Default  29.0000  29.0000
87    abrams tweet         102.000000  102.000000  Default  28.0000  28.0000
307   carrying democracy   102.000000  102.000000  Default  27.0000  27.0000
13    abrams carrying      102.000000  102.000000  Default  26.0000  26.0000
...               ...             ...         ...      ...      ...      ...
505   directing alternate  16.363805   17.196520   Topic10 -4.5852   2.6804 
1832  vaccine              20.447707   22.893559   Topic10 -4.3624   2.6170 
325   charge               18.374977   26.686032   Topic10 -4.4693   2.3569 
1619  stacey abrams        27.194777   402.998870  Topic10 -4.0773   0.0341 
1618  stacey               26.442234   411.351195  Topic10 -4.1053  -0.0145 

[467 rows x 6 columns], token_table=      Topic      Freq                  Term
term                                       
5     6      0.942010  abrams american     
12    8      0.896900  abrams brag         
13    3      0.989756  abrams carrying     
15    5      0.952409  abrams cheated      
17    6      0.964706  abrams consequential
...  ..           ...             ...      
1985  2      0.333945  year                
1985  5      0.589314  year                
1985  7      0.058931  year                
1990  5      0.941057  year torched        
1994  2      0.950282  yes                 

[624 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 8, 2, 4, 10, 7, 5, 3, 1, 9])

In [25]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(doc_term_matrix_1))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(doc_term_matrix_1))

Log Likelihood:  -167336.67622249882
Perplexity:  901.0673248447437


In [26]:
from sklearn.model_selection import GridSearchCV

# Define Search Param
search_params = {'n_components': [5, 8, 10, 12], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(doc_term_matrix_1)

GridSearchCV(estimator=LatentDirichletAllocation(),
             param_grid={'learning_decay': [0.5, 0.7, 0.9],
                         'n_components': [5, 8, 10, 12]})

In [27]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(doc_term_matrix_1))

Best Model's Params:  {'learning_decay': 0.9, 'n_components': 5}
Best Log Likelihood Score:  -39593.25449893281
Model Perplexity:  1000.9217320781232


In [30]:
# 2nd LDA model
lda_model_2 = LatentDirichletAllocation(n_components = 5, max_iter=10, learning_method='online', learning_decay=0.9)
lda_Z_2 = lda_model_2.fit_transform(doc_term_matrix_1)


pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model_2, doc_term_matrix_1, tfidfconverter, mds='tsne')
panel

pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model_2, doc_term_matrix_1, tfidfconverter, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
4      150.222153 -72.269554   1       1        21.643622
1      51.163364  -159.968018  2       1        21.014080
2     -73.144554   21.523815   3       1        20.851914
3     -66.410133  -110.606438  4       1        19.003975
0      54.417324   12.443444   5       1        17.486409, topic_info=                    Term        Freq       Total Category  logprob  loglift
306   carrying            114.000000  114.000000  Default  30.0000  30.0000
307   carrying democracy  113.000000  113.000000  Default  29.0000  29.0000
13    abrams carrying     113.000000  113.000000  Default  28.0000  28.0000
1800  tweet               118.000000  118.000000  Default  27.0000  27.0000
87    abrams tweet        109.000000  109.000000  Default  26.0000  26.0000
...            ...               ...         ...      ...      ...      ...
753   governor            30.696390   57.524172   Topic5  -4.9424   1.1157 
998   like                30.210268   84.631838   Topic5  -4.9584   0.7136 
662   fucking             25.746826   44.502157   Topic5  -5.1183   1.1965 
1985  year                25.737107   50.023637   Topic5  -5.1186   1.0792 
1647  state               26.438587   194.546927  Topic5  -5.0917  -0.2521 

[252 rows x 6 columns], token_table=      Topic      Freq                  Term
term                                       
13    5      0.995453  abrams carrying     
15    5      0.952590  abrams cheated      
17    3      0.966531  abrams consequential
33    4      0.989106  abrams goddess      
61    1      0.982987  abrams raphael      
...  ..           ...             ...      
1952  3      0.508148  work                
1952  5      0.034646  work                
1985  2      0.119943  year                
1985  3      0.359830  year                
1985  5      0.519754  year                

[322 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 3, 4, 1])